In [ ]:
import h5py    
import numpy as np 
import matplotlib.pyplot as plt
from pyjet import cluster,DTYPE_PTEPM
import pandas as pd

# Dataset loading

Notebook derived from https://github.com/lhcolympics2020/parsingscripts. We need the downloaded datasets from the LHC Olympics.

## R&D dataset

In [ ]:
fn_RandD = "events_anomalydetection_v2.h5"

## BB1 dataset

In [ ]:
fn_bb1 = "events_LHCO2020_BlackBox1.h5"
fn_key_bb1 = "events_LHCO2020_BlackBox1.masterkey"

In [ ]:
fn_2_bb1 = []
with open(fn_key_bb1, 'r') as f:# 'r' = read
    for nline, line in enumerate(f):
        if nline > 99:
            pass
#             break
        fn_2_bb1.append(float(line))
fn_2_bb1=np.array(fn_2_bb1).reshape(-1,1)

# Dataset processing

We turn the dataset into two arrays: $x$ containing the invariant masses and NSubjetiness of the two leading jets and $y$ containing the invariant mass of the event.

## R&D

In [ ]:
Nbkg = 1000000
Nsig = 100000
Ndata = Nbkg+Nsig
batch_size = 1000

In [ ]:
Ndata/batch_size

In [ ]:
#Now, let's cluster some jets!

# I create empty dictionaries to group bkg and signal

pT1 = {}
pT2 = {}

deltaPhi12 = {}
deltaEta12 = {}

mj1 = {}
mj2 = {}

tau21_1 = {}
tau21_2 = {}

tau31_1 = {}
tau31_2 = {}

tau32_1 = {}
tau32_2 = {}

alljets = {}

mjj={}

# I fill the dictionary with empty lists that I'll fill
for mytype in ['background','signal']:
    pT1[mytype]=[]
    pT2[mytype]=[]
    
    deltaPhi12[mytype]=[]
    deltaEta12[mytype]=[]
    
    mj1[mytype] = []
    mj2[mytype] = []
    
    tau21_1[mytype] = []
    tau21_2[mytype] = []
    
    tau31_1[mytype] = []
    tau31_2[mytype] = []

    tau32_1[mytype] = []
    tau32_2[mytype] = []
    
    alljets[mytype]=[]
    
    mjj[mytype]=[]

There are two cells here in case the user does not want to process everything and wants a subset of Background and of Signal

In [ ]:
for ii in range(int(Nbkg/batch_size)+1):
    start = ii*batch_size
    stop = (ii+1)*batch_size
    
    events_combined = np.array(pd.read_hdf(fn_RandD,start=start,stop=stop))
                                
    for mytype in ['background']:
        for i in range(np.shape(events_combined)[0]):
            if (i%1000==0):
#                 print(mytype,i)
                pass
            # get the true label of the event
            issignal = events_combined[i,2100]
            # check whether we filling the appropriate dictionary
            if (mytype=='background' and issignal):
                continue
            elif (mytype=='signal' and issignal==0):
                 continue
            #create a pseudo_jet input for the pyjet implementation
            pseudojets_input = np.zeros(len([x for x in events_combined[i,::3] if x > 0]), dtype=DTYPE_PTEPM)
            for j in range(700):
                if (events_combined[i][j*3]>0):
                    pseudojets_input[j]['pT'] = events_combined[i,j*3]
                    pseudojets_input[j]['eta'] = events_combined[i,j*3+1]
                    pseudojets_input[j]['phi'] = events_combined[i,j*3+2]
                    pass
                pass
            # cluster into jets
            sequence = cluster(pseudojets_input, R=1.0, p=-1)
            jets = sequence.inclusive_jets(ptmin=20)
            # get NSubjetiness (with homemade suboptimal algorithm)
            taus = np.array([nsubjet(jets[0],1.0,1.0),nsubjet(jets[0],1.0,2.0),nsubjet(jets[0],1.0,3.0),nsubjet(jets[1],1.0,1.0),nsubjet(jets[1],1.0,2.0),nsubjet(jets[1],1.0,3.0)])
            if taus[0]==0.0 or taus[3]==0.0 or taus[1]==0.0 or taus[4]==0.0:
                continue
            vec = np.array([taus[1]/taus[0], taus[4]/taus[3],taus[2]/taus[0], taus[5]/taus[3],taus[2]/taus[1], taus[5]/taus[4]])
            
            # order by mass
    
            if jets[0].mass >= jets[1].mass:
                ind1 = 0
                ind2 = 1
            else:
                ind1 = 1
                ind2 = 0
                
            # start saving 
            pT1[mytype] += [jets[ind1].pt]
            pT2[mytype] += [jets[ind2].pt]
            
            deltaPhi12[mytype] += [jets[ind1].phi-jets[ind2].phi]
            deltaEta12[mytype] += [jets[ind1].eta-jets[ind2].eta]
            
            mj1[mytype] += [jets[ind1].mass]
            mj2[mytype] += [jets[ind2].mass]
            
            tau21_1[mytype] +=[vec[ind1]]
            tau21_2[mytype] +=[vec[ind2]]

            tau31_1[mytype] +=[vec[2+ind1]]
            tau31_2[mytype] +=[vec[2+ind2]]
            
            tau32_1[mytype] +=[vec[4+ind1]]
            tau32_2[mytype] +=[vec[4+ind2]]
            
            alljets[mytype] += [jets]
            E = jets[0].e+jets[1].e
            px = jets[0].px+jets[1].px
            py = jets[0].py+jets[1].py
            pz = jets[0].pz+jets[1].pz
            mjj[mytype]+=[(E**2-px**2-py**2-pz**2)**0.5]
            pass

In [ ]:
for ii in range(int(Nsig/batch_size)+1):
    start = ii*batch_size
    stop = (ii+1)*batch_size
    
    events_combined = np.array(pd.read_hdf(fn_RandD,start=1000000+start,stop=1000000+stop))
    
    for mytype in ['signal']:
        for i in range(np.shape(events_combined)[0]):
            if (i%1000==0):
#                 print(mytype,i)
                pass
            # get the true label of the event
            issignal = events_combined[i,2100]
            # check whether we filling the appropriate dictionary
            if (mytype=='background' and issignal):
                continue
            elif (mytype=='signal' and issignal==0):
                 continue
            #create a pseudo_jet input for the pyjet implementation
            pseudojets_input = np.zeros(len([x for x in events_combined[i,::3] if x > 0]), dtype=DTYPE_PTEPM)
            for j in range(700):
                if (events_combined[i][j*3]>0):
                    pseudojets_input[j]['pT'] = events_combined[i,j*3]
                    pseudojets_input[j]['eta'] = events_combined[i,j*3+1]
                    pseudojets_input[j]['phi'] = events_combined[i,j*3+2]
                    pass
                pass
            # cluster into jets
            sequence = cluster(pseudojets_input, R=1.0, p=-1)
            jets = sequence.inclusive_jets(ptmin=20)
            # get NSubjetiness (with homemade suboptimal algorithm)
            taus = np.array([nsubjet(jets[0],1.0,1.0),nsubjet(jets[0],1.0,2.0),nsubjet(jets[0],1.0,3.0),nsubjet(jets[1],1.0,1.0),nsubjet(jets[1],1.0,2.0),nsubjet(jets[1],1.0,3.0)])
            if taus[0]==0.0 or taus[3]==0.0 or taus[1]==0.0 or taus[4]==0.0:
                continue
            vec = np.array([taus[1]/taus[0], taus[4]/taus[3],taus[2]/taus[0], taus[5]/taus[3],taus[2]/taus[1], taus[5]/taus[4]])
            
            # order by mass
    
            if jets[0].mass >= jets[1].mass:
                ind1 = 0
                ind2 = 1
            else:
                ind1 = 1
                ind2 = 0
                
            # start saving 
            pT1[mytype] += [jets[ind1].pt]
            pT2[mytype] += [jets[ind2].pt]
            
            deltaPhi12[mytype] += [jets[ind1].phi-jets[ind2].phi]
            deltaEta12[mytype] += [jets[ind1].eta-jets[ind2].eta]
            
            mj1[mytype] += [jets[ind1].mass]
            mj2[mytype] += [jets[ind2].mass]
            
            tau21_1[mytype] +=[vec[ind1]]
            tau21_2[mytype] +=[vec[ind2]]

            tau31_1[mytype] +=[vec[2+ind1]]
            tau31_2[mytype] +=[vec[2+ind2]]
            
            tau32_1[mytype] +=[vec[4+ind1]]
            tau32_2[mytype] +=[vec[4+ind2]]
            
            alljets[mytype] += [jets]
            E = jets[0].e+jets[1].e
            px = jets[0].px+jets[1].px
            py = jets[0].py+jets[1].py
            pz = jets[0].pz+jets[1].pz
            mjj[mytype]+=[(E**2-px**2-py**2-pz**2)**0.5]
            pass

In [ ]:
y_together = np.hstack([mjj['background'],mjj['signal']])
labels_together = np.hstack([np.zeros(len(mjj['background'])),np.ones(len(mjj['signal']))])
x_together = np.vstack([np.vstack([mj1['background'],mj2['background'],tau21_1['background'],tau21_2['background']]).T,np.vstack([mj1['signal'],mj2['signal'],tau21_1['signal'],tau21_2['signal']]).T])

In [ ]:
np.save('y_RandD.npy',y_together)
np.save('labels_RandD.npy',labels_together)
np.save('x_RandD.npy',x_together)

## BB1

In [ ]:
Ndata = 1000000
batch_size = 1000

In [ ]:
#Now, let's cluster some jets!

# I create empty dictionaries to group bkg and signal

pT1 = {}
pT2 = {}

deltaPhi12 = {}
deltaEta12 = {}

mj1 = {}
mj2 = {}

tau21_1 = {}
tau21_2 = {}

tau31_1 = {}
tau31_2 = {}

tau32_1 = {}
tau32_2 = {}

alljets = {}

mjj={}

# I fill the dictionary with empty lists that I'll fill
for mytype in ['background','signal']:
    pT1[mytype]=[]
    pT2[mytype]=[]
    
    deltaPhi12[mytype]=[]
    deltaEta12[mytype]=[]
    
    mj1[mytype] = []
    mj2[mytype] = []
    
    tau21_1[mytype] = []
    tau21_2[mytype] = []
    
    tau31_1[mytype] = []
    tau31_2[mytype] = []

    tau32_1[mytype] = []
    tau32_2[mytype] = []
    
    alljets[mytype]=[]
    
    mjj[mytype]=[]

In [ ]:
for ii in range(int(Ndata/batch_size)+1):
    start = ii*batch_size
    stop = (ii+1)*batch_size
    
    events_combined = np.hstack([np.array(pd.read_hdf(fn_BB1,start=start,stop=stop)),fn_2_bb1[start:stop]])
                                
    for mytype in ['background','signal']:
        for i in range(np.shape(events_combined)[0]):
            if (i%1000==0):
#                 print(mytype,i)
                pass
            # get the true label of the event
            issignal = events_combined[i,2100]
            # check whether we filling the appropriate dictionary
            if (mytype=='background' and issignal):
                continue
            elif (mytype=='signal' and issignal==0):
                 continue
            #create a pseudo_jet input for the pyjet implementation
            pseudojets_input = np.zeros(len([x for x in events_combined[i,::3] if x > 0]), dtype=DTYPE_PTEPM)
            for j in range(700):
                if (events_combined[i][j*3]>0):
                    pseudojets_input[j]['pT'] = events_combined[i,j*3]
                    pseudojets_input[j]['eta'] = events_combined[i,j*3+1]
                    pseudojets_input[j]['phi'] = events_combined[i,j*3+2]
                    pass
                pass
            # cluster into jets
            sequence = cluster(pseudojets_input, R=1.0, p=-1)
            jets = sequence.inclusive_jets(ptmin=20)
            # get NSubjetiness (with homemade suboptimal algorithm)
            taus = np.array([nsubjet(jets[0],1.0,1.0),nsubjet(jets[0],1.0,2.0),nsubjet(jets[0],1.0,3.0),nsubjet(jets[1],1.0,1.0),nsubjet(jets[1],1.0,2.0),nsubjet(jets[1],1.0,3.0)])
            if taus[0]==0.0 or taus[3]==0.0 or taus[1]==0.0 or taus[4]==0.0:
                continue
            vec = np.array([taus[1]/taus[0], taus[4]/taus[3],taus[2]/taus[0], taus[5]/taus[3],taus[2]/taus[1], taus[5]/taus[4]])
            
            # order by mass
    
            if jets[0].mass >= jets[1].mass:
                ind1 = 0
                ind2 = 1
            else:
                ind1 = 1
                ind2 = 0
                
            # start saving 
            pT1[mytype] += [jets[ind1].pt]
            pT2[mytype] += [jets[ind2].pt]
            
            deltaPhi12[mytype] += [jets[ind1].phi-jets[ind2].phi]
            deltaEta12[mytype] += [jets[ind1].eta-jets[ind2].eta]
            
            mj1[mytype] += [jets[ind1].mass]
            mj2[mytype] += [jets[ind2].mass]
            
            tau21_1[mytype] +=[vec[ind1]]
            tau21_2[mytype] +=[vec[ind2]]

            tau31_1[mytype] +=[vec[2+ind1]]
            tau31_2[mytype] +=[vec[2+ind2]]
            
            tau32_1[mytype] +=[vec[4+ind1]]
            tau32_2[mytype] +=[vec[4+ind2]]
            
            alljets[mytype] += [jets]
            E = jets[0].e+jets[1].e
            px = jets[0].px+jets[1].px
            py = jets[0].py+jets[1].py
            pz = jets[0].pz+jets[1].pz
            mjj[mytype]+=[(E**2-px**2-py**2-pz**2)**0.5]
            pass

In [ ]:
y_together = np.hstack([mjj['background'],mjj['signal']])
labels_together = np.hstack([np.zeros(len(mjj['background'])),np.ones(len(mjj['signal']))])
x_together = np.vstack([np.vstack([mj1['background'],mj2['background'],tau21_1['background'],tau21_2['background']]).T,np.vstack([mj1['signal'],mj2['signal'],tau21_1['signal'],tau21_2['signal']]).T])

In [ ]:
np.save('y_BB1.npy',y_together)
np.save('labels_BB1.npy',labels_together)
np.save('x_BB1.npy',x_together)

# Plots

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(mj1['background'], bins=50, facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(mj1['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$m_{j_{1}}$ [GeV]')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.show()
# plt.savefig("mj1.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(np.array(mj1['background'])-np.array(mj2['background']), bins=50, facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(np.array(mj1['signal'])-np.array(mj2['signal']), bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\Delta m_{j}$ [GeV]')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.show()
# plt.savefig("deltamj.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau21_1['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau21_1['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{21,1}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
# plt.savefig("tau21_1.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau21_2['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau21_2['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{21,2}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
# plt.savefig("tau21_2.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau31_1['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau31_1['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{31,1}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
# plt.savefig("tau31_1.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau31_2['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau31_2['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{31,2}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
plt.savefig("tau31_2.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau32_1['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau32_1['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{32,1}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
# plt.savefig("tau32_1.pdf")

In [ ]:
#Let's make some very simple plots.
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(tau32_2['background'], bins=np.linspace(0,1), facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(tau32_2['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$\tau_{32,2}$')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.show()
# plt.savefig("tau32_2.pdf")

In [ ]:
plt.scatter(mj1['background'],deltamj['background'])
plt.scatter(mj1['signal'],deltamj['signal'])

In [ ]:
plt.scatter(mj1['background'],tau21_1['background'])
plt.scatter(mj1['signal'],tau21_1['signal'])

In [ ]:
plt.scatter(mj1['background'],tau21_2['background'])
plt.scatter(mj1['signal'],tau21_2['signal'])

In [ ]:
plt.scatter(tau21_1['background'],tau21_2['background'])
plt.scatter(tau21_1['signal'],tau21_2['signal'])

In [ ]:
plt.scatter(deltamj['background'],tau21_1['background'])
plt.scatter(deltamj['signal'],tau21_1['signal'])

In [ ]:
plt.scatter(deltamj['background'],tau21_2['background'])
plt.scatter(deltamj['signal'],tau21_2['signal'])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
n,b,p = plt.hist(mjj['background'], bins=50, facecolor='r', alpha=0.2,label='background',density=True)
plt.hist(mjj['signal'], bins=b, facecolor='b', alpha=0.2,label='signal',density=True)
plt.xlabel(r'$m_{JJ}$ [GeV]')
plt.ylabel('Number of events')
plt.legend(loc='upper right')
plt.show()
# plt.savefig("mjj.pdf")